Парсер prodoctorov.ru.

Часть 2, собирает данные со страниц.

Последние правки: 2021-12-28

## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю данные ##

In [ ]:
# нужный город латиницей, транслитом как на сайте

need_city = input()

In [ ]:
# загружаю ссылки

work_links = pd.read_excel(file_path + f'prodoctorov_urls_{need_city}.xlsx')

In [ ]:
work_links.sample(3)

In [ ]:
# конвертирую в список

links = work_links.clinic_page.to_list()

In [ ]:
len(links)

### Код парсера ###

In [ ]:
data = [] # список для данных

for page_link in tqdm(links): # цикл для сбора ссылок
            
    driver.get(page_link)
    sleep(3)
    driver.implicitly_wait(5)
 
    # базовые данные

    # название
    clinic = driver.find_element_by_class_name('b-doctor-intro__title-name_margin-bottom_none').text

    # город
    city = driver.find_element_by_class_name('b-doctor-intro__title-sub_margin-bottom_xs').text

    # тип клиники
    clinic_type = driver.find_element_by_class_name('b-doctor-intro__title-type').text

    # текст аннотации
    annotation = driver.find_element_by_class_name('b-text-unit.b-text-unit_color_medium.b-text-unit_mb_sm').text

    # адрес и телефон беру из блока "Информация", без мусора

    # средние оценки
    scores = driver.find_elements_by_class_name('b-doctor-stats__stats-num') # достаю оценки

    temp = [] # разбираю
    for item in scores:
        if item.text != '':
            temp += [item.text]

        else:
            temp += ['none']

    clinic_score = temp[0] # оценка клиники
    doc_score = temp[1] # врача
    doc_exp_score = temp[2] # опыт врачей в среднем


    # количество отзывов
    try:
        review_number = (driver
                         .find_element_by_class_name('b-doctor-intro__rates-link')
                         .text.replace(' отзывов', '')
                        )

    except:
        review_number = 'none'

    # дата обновления 
    info_date = driver.find_element_by_class_name('b-doctor-intro__updated').text.replace('Обновлено ', '')

    # часы работы
    try: # кликаю на элемент
        driver.find_element_by_class_name('tooltipstered').click()

        # забираю данные
        work_hours = 'wrong'
        WebDriverWait(driver, 5).until(
                        EC.visibility_of_element_located((By.CLASS_NAME, 'b-lpu-schedule__list')))
        work_hours = driver.find_element_by_class_name('b-lpu-schedule__list').text.replace(' :\n', ': ').split('\n')

    except:
        work_hours = 'none'

    # оценки клиники
    try: # перехожу в раздел
        driver.find_element_by_class_name('b-doctor-intro__rates-link').click()
        sleep(1)

        # вытаскиваю данные
        reviews = driver.find_elements_by_class_name('b-doctor-rates__filter-number')

        temp = []  # разбираю данные в список
        for item in reviews:
            temp += [item.text]

        # записываю в переменные самым простым способом 
        good_reviews = temp[1]
        neutral_reviews = temp[2]
        bad_reviews = temp[3]

        # возвращаюсь назад
        driver.back()

    except:
        good_reviews = 0
        neutral_reviews = 0
        bad_reviews = 0


    # раздел "Информация"

    # перехожу в раздел
    driver.find_element_by_link_text('Информация').click()
    sleep(1)

    # забираю блок с информацией
    info_block = driver.find_element_by_class_name('b-clinic-profile')

    # ближайшие станции метро    
    try:
        metro = driver.find_elements_by_class_name('b-clinic-profile__metro-item')
        metro_stations = {}

        for station in metro:
            metro_stations[station.text.split('\n')[0]] = station.text.split('\n')[1].replace('(', '').replace(')', '')

    except:
        metro_stations = {}


    # официальная информация
    of_info = {}
    of_info_block = info_block.find_elements_by_class_name('b-clinic-profile__item') # выгружаю весь правый блок

    for i in of_info_block: # разбираю
        temp_text = i.text.split('\n')

        try:
            of_info[temp_text[0]] = temp_text[1]

        except:
            of_info['address'] = temp_text[0] 

    # записываю данные
    try: # адрес
        address = of_info['address']

    except:
        address = 'none'

    # телефон        
    try:
        phone = of_info['Телефон для записи']

    except:
        phone = 'none'

    # Юридическое название        
    try:
        of_name = of_info['Официальное название']

    except:
        of_name = 'none'


    # характеристики      
    features = info_block.find_elements_by_class_name('b-clinic-profile__feature') # вытаскиваю блок

    clinic_features = [] # разбираю данные
    for item in features:
        clinic_features += [item.text]

    # сайт
    try:
        web_site = info_block.find_element_by_tag_name('input[name="url"]').get_attribute('value')

    except:
        web_site = 'none'


    # описание
    clinic_description = info_block.find_element_by_class_name('b-clinic-profile__content').text

    # возвращаюсь назад
    driver.back()

    # связанные клиники
    related_clinics = []
    
    try:
        driver.find_element_by_tag_name('a[href="#related-lpus"]').click()
        sleep(1)

        related_clinics_block = driver.find_elements_by_class_name('b-related-lpus__header')

        for i in related_clinics_block:
            related_clinics += [i.text]

        # перехожу назад
        driver.back()

    except:
        pass

    # собираю цены

    # открываю вкладку с ценами
    try:
        driver.find_element_by_partial_link_text('Цены').click()
        sleep(1)

        try: # собираю цены на прием специалистов
            consultation_block = driver.find_elements_by_class_name('b-clinic-prices__list-item')
            consultation_price = {}

            for item in consultation_block:
                temp = item.text.split('\n')
                consultation_price[temp[0]] = temp[1].replace(' ₽', '')

        except:
            consultation_price = {}

        try:
            full_price = {}

            # беру блок с прайсом
            full_price_block = (driver
                                .find_elements_by_class_name('b-clinic-full-price__item.b-clinic-full-price__item_price-center'))

            for i in full_price_block: # вытаскиваю заголовок
                serv_title = i.find_element_by_class_name('b-clinic-full-price__item-title').text

                # вытаскиваю услуги и цены
                services = i.find_elements_by_class_name('b-clinic-full-price__service')
                serv_dict = {}

                for item_price in services:
                    serv = item_price.find_element_by_class_name('b-clinic-full-price__key').text
                    cost = item_price.find_element_by_class_name('b-clinic-full-price__value').text.replace('₽', '')

                    # кладу данные в словарь
                    serv_dict[serv] = cost
                    full_price[serv_title] = serv_dict

        except:
            full_price = {}

       # перехожу назад
        driver.back()

    except:
        consultation_price = {}
        full_price = {}


    # врачи

    # собираю количество врачей
    try:
        doc_number = (driver
                  .find_element_by_css_selector('h2.b-clinic-title')
                  .get_attribute('data-count')
                  .replace('\n', '').strip().replace(' врачей', ''))

    except:
        doc_number = 0

    try:

        # перехожу на вкладку врачей
        driver.find_element_by_class_name('b-clinic-employees__more.b-button.b-button_light').click()
        sleep(2)

        # забираю блок с врачами
        specialist_blocks = driver.find_elements_by_class_name('b-doctor-card')

        doc_number_2 = len(specialist_blocks)

        specialist = {}
        for i in specialist_blocks:

            # собираю словарь с характеристиками врача
            spec_dict = {}
            spec_dict['фио'] = i.get_attribute('data-doctor-fio')
            spec_dict['имя'] = i.find_element_by_class_name('b-doctor-card__name-surname').text
            spec_dict['пол'] = i.get_attribute('data-gender')
            spec_dict['стаж'] = i.get_attribute('data-experience')
            spec_dict['профиль'] = i.find_element_by_class_name('b-doctor-card__spec').text
            spec_dict['кол-во отзывов'] = i.find_element_by_class_name('b-profile-card').text
            spec_dict['spec_page'] = i.find_element_by_class_name('b-doctor-card__name-link').get_attribute('href')

            # кладу словарь в словарь
            specialist[i.get_attribute('data-doctor-id')] = spec_dict


    except:
        try:
            # забираю блок с врачами
            specialist_blocks = driver.find_elements_by_class_name('b-doctor-card')
            doc_number_2 = len(specialist_blocks)
            specialist = {}

            for i in specialist_blocks:

                # собираю словарь с характеристиками врача
                spec_dict = {}
                spec_dict['фио'] = i.get_attribute('data-doctor-fio')
                spec_dict['имя'] = i.find_element_by_class_name('b-doctor-card__name-surname').text
                spec_dict['пол'] = i.get_attribute('data-gender')
                spec_dict['стаж'] = i.get_attribute('data-experience')
                spec_dict['профиль'] = i.find_element_by_class_name('b-doctor-card__spec').text
                spec_dict['кол-во отзывов'] = i.find_element_by_class_name('b-profile-card').text
                spec_dict['spec_page'] = i.find_element_by_class_name('b-doctor-card__name-link').get_attribute('href')

                # кладу словарь в словарь
                specialist[i.get_attribute('data-doctor-id')] = spec_dict

        except:
            specialist = {}

    data.append([clinic, of_name, clinic_type,
                 city, address, metro_stations,
                 work_hours, phone, web_site,
                 clinic_score, doc_score, doc_exp_score,
                 review_number, good_reviews, neutral_reviews, bad_reviews,
                 clinic_features,
                 related_clinics,
                 doc_number, doc_number_2, specialist,
                 consultation_price, full_price,

                 of_info, annotation, clinic_description,
                 info_date, page_link                 
                ])

In [ ]:
# записываю данные в датафрейм

head_list = ['clinic', 'of_name', 'clinic_type',
             'city', 'address', 'metro_stations',
             'work_hours', 'phone', 'web_site',
             'clinic_score', 'doc_score', 'doc_exp_score',
             'review_number', 'good_reviews', 'neutral_reviews', 'bad_reviews',
             'clinic_features',
             'related_clinics',
             'doc_number', 'doc_number_2', 'specialist',
             'consultation_price', 'full_price',
             'of_info', 'annotation', 'clinic_description',
             'info_date', 'clinic_page']

work = pd.DataFrame(data, columns=head_list)


In [ ]:
work.shape

In [ ]:
work.sample(2)

In [ ]:
work.to_csv(file_path + f'prodoctorov_info_{need_city}.csv', encoding='utf8', index=False)

In [ ]:
driver.quit()